# Generate example datasets

In [1]:
import numpy as np
from nexusformat.nexus import *

In [2]:
std_devs = [0.03, 0.05, 0.08]

for idx, T in enumerate(['15', '100', '300']):
    # Create an array
    array_size = (100, 150, 200)
    array = np.zeros(array_size)

    # Create coordinates for each point in the array
    x, y, z = np.meshgrid(np.linspace(0, 1, array_size[0]),
                          np.linspace(0, 1, array_size[1]),
                          np.linspace(0, 1, array_size[2]),
                          indexing='ij')

    # Create a 3x3x3 grid of evenly spaced Gaussians
    grid_size = (3, 3, 3)
    grid_center = (0, 0, 0)
    std_dev = std_devs[idx]

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            for k in range(grid_size[2]):
                mean = ((i / (grid_size[0] - 1)),
                        (j / (grid_size[1] - 1)),
                        (k / (grid_size[2] - 1)))
                amplitude = 1 / (np.sqrt(2 * np.pi) * std_dev)
                gaussian = amplitude * np.exp(-((x - grid_center[0] - mean[0]) ** 2 +
                                                (y - grid_center[1] - mean[1]) ** 2 +
                                                (z - grid_center[2] - mean[2]) ** 2) / (2 * std_dev ** 2))
                array += gaussian

    # Normalize the array
    # array /= np.max(array)

    H = NXfield(np.linspace(-1, 1, array.shape[0]), name='H')
    K = NXfield(np.linspace(-1, 1, array.shape[1]), name='K')
    L = NXfield(np.linspace(-1, 1, array.shape[2]), name='L')

    data = NXdata(NXfield(array, name='counts'), (H, K, L))

    out = NXroot()
    out['entry'] = NXentry()
    out['entry']['data'] = data
    nxsave('example_data/sample_name/' + T + '/example_hkli.nxs', out)
